In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [4]:
train = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


In [94]:
# data_augmentation = keras.Sequential(
# [
#     layers.RandomFlip("horizontal"),
#     layers.RandomRotation(0.1),
#     layers.RandomZoom(0.2),
#     layers.RandomContrast(0.2),
#     layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
# ] 
# )

In [5]:
inputs = keras.Input(shape=(100, 100, 3))

# x = data_augmentation(inputs)
x = layers.Conv2D(16, 5, activation='relu')(inputs)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(16, 5, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.MaxPooling2D(2)(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)

x = layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.008), activation='relu')(x)

x = layers.Dropout(0.3)(x)

outputs = layers.Dense(6, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 16)        1216      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 16)        6416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 22, 22, 16)       0         
 2D)                                                             
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 11, 11, 16)       0     

In [150]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/zenodo_5.h5",
        monitor="val_loss",
        save_best_only=True
    )
]

In [151]:
history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-01 08:41:36.382445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


104/105 [============================>.] - ETA: 0s - loss: 2.9110 - accuracy: 0.1517

2022-08-01 08:41:39.026663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 4s 26ms/step - loss: 2.9012 - accuracy: 0.1509 - val_loss: 1.8789 - val_accuracy: 0.1667
Epoch 2/40
105/105 [==============================] - 2s 23ms/step - loss: 2.2571 - accuracy: 0.1524 - val_loss: 1.8199 - val_accuracy: 0.1667
Epoch 3/40
105/105 [==============================] - 3s 24ms/step - loss: 1.9762 - accuracy: 0.1583 - val_loss: 1.8613 - val_accuracy: 0.1819
Epoch 4/40
105/105 [==============================] - 2s 23ms/step - loss: 1.6754 - accuracy: 0.3185 - val_loss: 1.2543 - val_accuracy: 0.5569
Epoch 5/40
105/105 [==============================] - 2s 23ms/step - loss: 1.2277 - accuracy: 0.5640 - val_loss: 1.1446 - val_accuracy: 0.5875
Epoch 6/40
105/105 [==============================] - 3s 24ms/step - loss: 0.9565 - accuracy: 0.6604 - val_loss: 0.8379 - val_accuracy: 0.7597
Epoch 7/40
105/105 [==============================] - 2s 23ms/step - loss: 0.7905 - accuracy: 0.7301 - val_loss: 0.6535 - val_accuracy: 0.7917
Epoch 8/40

In [152]:
model.evaluate(test)

23/23 [==============================] - 0s 14ms/step - loss: 1.1438 - accuracy: 0.8486


[1.1437840461730957, 0.8486111164093018]

In [155]:
# keras.models.save_model(model, "./models/zenodo_3.h5")

load_model = keras.models.load_model("models/zenodo_5.h5")
load_model.evaluate(test)

 4/23 [====>.........................] - ETA: 0s - loss: 0.5667 - accuracy: 0.8203

2022-08-01 08:44:12.758027: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 16ms/step - loss: 0.5823 - accuracy: 0.8139


[0.5823264718055725, 0.8138889074325562]